In [1]:
from tqdm.auto import tqdm, trange
import jax.numpy as jnp
import numpy as np
import jax

In [2]:
import jax_smi
jax_smi.initialise_tracking()

In [3]:
# input_size = 2**19 + 2**18
input_size = 2**18
hidden_size = 2**14
w_chunk = 2**12
w_size = 2**20
k = 64
encoder_scale = 300
# n_dp, n_mp = 1, 4
n_dp, n_mp = 4, 1
# n_dp, n_mp = 1, 1
input_size = input_size * n_dp

In [4]:
one_v4_chip_flops = 275 * 1e12  # https://cloud.google.com/tpu/docs/v4
v4_8_flops = one_v4_chip_flops * 4
chunk_flops = input_size * hidden_size * w_chunk * n_mp
task_flops = input_size * hidden_size * w_size
print("For chunk:", chunk_flops / v4_8_flops)
print("For task:", task_flops / v4_8_flops)

For chunk: 0.06397158561605819
For task: 16.376725917710896


In [5]:
devices = np.asarray(jax.local_devices()).reshape(n_dp, n_mp)
mesh = jax.sharding.Mesh(devices, ("dp", "mp"))

cpu = jax.devices("cpu")[0]
to_cpu = lambda x: jax.device_put(x, cpu)

In [6]:
from splatmul import matmat
with jax.default_device(cpu):
    weight_enc, weight_dec = map(lambda x: jnp.asarray(x).view(jnp.bfloat16), matmat(w_size, hidden_size))

In [7]:
from splatmul import splatsplat, splatmat
adam_state_encoder = splatsplat(w_size, hidden_size,
                        1e-4, 0.9, 0.999, 1e-8, 16)
adam_state_decoder = splatsplat(w_size, hidden_size,
                        1e-4, 0.9, 0.999, 1e-8, 16)

In [8]:
import ctypes


def get_writeable_bit():
    max_arr_size = 128
    test_arr = np.array(1, dtype=np.uint16)
    ptr = id(test_arr)
    str_a = ctypes.string_at(ptr, max_arr_size)
    test_arr.setflags(write=0)
    str_b = ctypes.string_at(ptr, max_arr_size)
    writeable_bit = None
    for i in range(max_arr_size):
        if str_a[i] != str_b[i]:
            byte_a, byte_b = bin(str_a[i])[2:].zfill(8), bin(str_b[i])[2:].zfill(8)
            for j in range(8):
                if byte_a[j] != byte_b[j]:
                    writeable_bit = i * 8 + j
    writeable_byte = writeable_bit // 8, writeable_bit % 8
    return writeable_byte

def transmogrify(x):
    byte, bit = get_writeable_bit()
    ptr = id(x)
    current = ctypes.string_at(ptr + byte, 1)
    target = current[0] | (1 << (7-bit))
    ctypes.memset(ptr + byte, target, 1)
    return x


In [9]:
import gc


# key_sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec("dp"))
key_sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec())
data_sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec("dp", None))
weight_sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec(None, "mp"))
topk_sharding = jax.sharding.NamedSharding(mesh, jax.sharding.PartitionSpec("dp", "mp"))
# single_mesh = jax.sharding.Mesh(devices[:1, :], ("dp", "mp"))
# single_sharding = jax.sharding.NamedSharding(single_mesh, jax.sharding.PartitionSpec(None, "mp"))

def split(key):
    a, b = jax.random.split(key)
    return a, b

def gen_inputs(input_key):
    # input_key, subkey = split(input_key)
    input_key, subkey = input_key + 1, input_key
    
    # inputs = jax.vmap(lambda x: jax.random.randint(x, (input_size // mesh.shape["dp"], hidden_size), -100, 100, jnp.int8))(subkey).reshape(input_size, hidden_size)
    # inputs = jax.random.randint(subkey, (input_size, hidden_size), -100, 100, jnp.int8)
    # inputs = jax.random.bits(subkey, shape=(input_size, hidden_size), dtype=jnp.uint8)
    inputs = jax.lax.broadcasted_iota(jnp.int4, (input_size, hidden_size), 0).astype(jnp.int8) + jax.lax.broadcasted_iota(jnp.int4, (input_size, hidden_size), 1).astype(jnp.int8) + subkey
    # inputs = jnp.ones((input_size, hidden_size), jnp.int8)
    return inputs, input_key
gen_inputs_jit = jax.jit(gen_inputs, in_shardings=(key_sharding,), out_shardings=(data_sharding, key_sharding), donate_argnums=(0,))

def matmul_topk(inputs, weight_chunk, old_weights, old_indices, offset):
    output_chunk = inputs @ weight_chunk
    # weights, indices = jax.lax.top_k(output_chunk, k=k)
    # weights, indices = jax.lax.approx_max_k(output_chunk, k=k)
    weights, indices = jax.lax.approx_max_k(output_chunk.astype(jnp.bfloat16), k=k, recall_target=0.8)
    weights = weights * (encoder_scale / 127.5)
    indices += offset
    # weights, indices = jax.lax.approx_max_k(output_chunk, k=k, recall_target=0.5)
    # weights, indices = jax.lax.approx_max_k(output_chunk, k=k, recall_target=0.25)
    
    # _, indices = jax.lax.approx_max_k(output_chunk[:, :k * 4], k=k)
    # _, indices = jax.lax.approx_max_k(output_chunk[:, :k], k=k)
    # weights = output_chunk.reshape(output_chunk.shape[0], k, -1).sum(-1)
    # weights, indices = jax.lax.top_k(output_chunk, k=k)
    
    if old_weights is None or old_indices is None:
        return weights, indices
    else:
        # replace_mask = weights > old_weights
        # return jnp.where(replace_mask, weights, old_weights), jnp.where(replace_mask, indices, old_indices)

        new_full_weights = jnp.concatenate((weights, old_weights), axis=1)
        new_full_indices = jnp.concatenate((indices, old_indices), axis=1)
        
        # over_indices = jnp.argsort(new_full_weights)[:, -k:]
        # return jnp.take_along_axis(new_full_weights, over_indices, axis=1), jnp.take_along_axis(new_full_indices, over_indices, axis=1)
        
        _, over_indices = jax.lax.top_k(new_full_weights, k=k)
        return jnp.take_along_axis(new_full_weights, over_indices, axis=1), jnp.take_along_axis(new_full_indices, over_indices, axis=1)

matmul_topk_jit = jax.jit(matmul_topk, in_shardings=(data_sharding, weight_sharding, topk_sharding, topk_sharding, None), out_shardings=(topk_sharding, topk_sharding), donate_argnums=(2, 3))

# weights_0_put = jax.device_put(weights_enc[0], weight_sharding)
# send_weights = lambda _: weights_0_put
def send_weights(weights):
    # weights = jax.device_put(weights, single_sharding)
    return jax.device_put(weights, weight_sharding)

In [10]:
import splatmul
import gc


def sparse_matmul(weights, indices, encoder_weight, decoder_weight, input_embeds, target_embeds):
    with jax.default_device(cpu):
        weights = np.asarray(weights.view(jnp.uint16))
        indices = np.asarray(indices.astype(jnp.uint32))
        decoder_weight = np.asarray(decoder_weight.view(jnp.uint16))
        encoder_weight = np.asarray(encoder_weight.view(jnp.uint16))
        result = splatmul.splatmul(weights, indices, decoder_weight)
        gc.collect()
        grads = None, None
        input_embeds = np.asarray(input_embeds)
        target_embeds = np.asarray(target_embeds)
        print(result.shape, result.dtype)
        encoder_grads, decoder_grads = splatmul.matsplat(input_embeds, target_embeds, encoder_weight, decoder_weight, weights, indices, result)
        grads = (jnp.asarray(encoder_grads.view(jnp.bfloat16)), jnp.asarray(decoder_grads.view(jnp.bfloat16)))
        gc.collect()
        return result, grads

In [11]:
from threading import Thread
from queue import Queue

def sparse_matmul_async(weights, indices, past_embeds=None):
    out_queue = Queue()
    def worker():
        out_queue.put(sparse_matmul(weights, indices, weight_enc, weight_dec, past_embeds, past_embeds))
    thread = Thread(target=worker)
    thread.start()
    return out_queue

: 

In [12]:
import threading
import queue
import time


def matmul_trial(inputs):
    with jax.default_device(cpu):
        chunk = weight_enc[:w_chunk * n_mp].T
    current_chunk = send_weights(chunk)
    weights, indices = None, None
    bar = trange(0, w_size, w_chunk * n_mp, postfix=f"Encoder forward pass")
    offset = 0
    for chunk_start in bar:
        with jax.default_device(cpu):
            weight_enc[chunk_start:chunk_start + w_chunk * n_mp].T
        next_chunk = send_weights(chunk)
        weights, indices = matmul_topk_jit(inputs, current_chunk, weights, indices, offset)
        offset += w_chunk * n_mp
        current_chunk = next_chunk
        gc.collect()
    return weights, indices


def trial(save_encodings=False):
    global saved_encodings
    gc.collect()
    input_key = 0  # jax.random.split(jax.random.key(0), mesh.shape["dp"])
    encodings, inputs, decoder_outputs, encoder_grads, decoder_grads = None, None, None, None, None
    for _ in trange(100, postfix="Measuring forward speed..."):
        gc.collect()
        if encodings is not None:
            past_inputs = jax.device_put(inputs, cpu)
            decoder_outputs = sparse_matmul_async(encodings[0], encodings[1], past_embeds=past_inputs)
            del past_inputs, inputs
        inputs, input_key = gen_inputs_jit(input_key)
        weights, indices = matmul_trial(inputs)
        if decoder_outputs is not None:
            print("Computing decoder/grads...")
            decoder_start = time.time()
            with jax.default_device(cpu):
                decoder_outputs, (encoder_grads, decoder_grads) = jax.block_until_ready(decoder_outputs.get())
            decoder_end = time.time()
            print("Decoder/grad time:", decoder_end - decoder_start)
            del decoder_outputs
            gc.collect()
        print("Waiting for encoder...")
        before_compute = time.time()
        weights, indices = weights.block_until_ready(), indices.block_until_ready()
        after_compute = time.time()
        print("Encoder time:", (after_compute - before_compute))
        weights, indices = to_cpu(weights), to_cpu(indices)
        encodings = weights, indices
        del weights, indices
        if encoder_grads is not None and decoder_grads is not None:
            with jax.default_device(cpu):
                print("Updating encoder weights...")
                weight_dec_np = transmogrify(np.asarray(weight_dec.view(jnp.uint16)))
                splatmat(adam_state_decoder, decoder_grads, weight_dec_np)
                print("Updating decoder weights...")
                weight_enc_np = transmogrify(np.asarray(weight_enc.view(jnp.uint16)))
                splatmat(adam_state_encoder, encoder_grads, weight_enc_np)
                del encoder_grads, decoder_grads, weight_enc_np, weight_dec_np
        gc.collect()
        if save_encodings:
            saved_encodings = encodings
            break

trial()

  0%|          | 0/100 [00:00<?, ?it/s, Measuring forward speed...]

  0%|          | 0/256 [00:00<?, ?it/s, Encoder forward pass]

Waiting for encoder...
Encoder time: 17.734767198562622


  0%|          | 0/256 [00:00<?, ?it/s, Encoder forward pass]

(1048576, 16384) int8
Computing decoder/grads...


In [ ]:
# 1/0

In [ ]:
weights, indices = saved_encodings

In [ ]:
# # sort on CPU is slow
# import jax.experimental.sparse


# def sort_encodings(weights, indices):
#     index_array = indices.ravel().astype(jnp.uint32)
#     weight_array = weights.ravel().astype(jnp.bfloat16)
#     indices_to_sort = jnp.argsort(index_array)
#     sorted_indices = index_array[indices_to_sort]
#     sorted_weights = weight_array[indices_to_sort]
#     inverse_indices = jnp.argsort(indices_to_sort)
#     bcoo = jax.experimental.sparse.BCOO((sorted_weights, jnp.stack((inverse_indices, sorted_indices), axis=-1).reshape(-1, 2)), shape=weights.shape)
#     return bcoo

# sort_encodings_jit = jax.jit(sort_encodings, backend="cpu")
# %time bcoo = sort_encodings_jit(weights, indices).block_until_ready();

In [ ]:
# # Numba doesn't support bfloat16
# import ml_dtypes
# import numba as nb

# index_array_np = np.asarray(indices.ravel().astype(np.uint32))
# weight_array_np = np.asarray(weights.ravel().view(jnp.float16))
# %time indices_to_sort = np.argsort(index_array_np);
# %time inverse_indices = np.argsort(indices_to_sort);
# %time sorted_indices = index_array_np[indices_to_sort];
# %time sorted_weights = weight_array_np[indices_to_sort];

# @nb.njit
# def coo_matmul(inverse_indices, sorted_indices, sorted_weights, weights_dec):
#     N = inverse_indices.shape[0] // k
#     D = weights_dec.shape[1]
#     out = np.zeros((N, D), dtype=ml_dtypes.bfloat16)
#     for i in range(len(inverse_indices)):
#         out[inverse_indices[i] // k] += (sorted_weights[i].view(ml_dtypes.bfloat16).astype(np.float32) * weights_dec[sorted_indices[i]].view(ml_dtypes.bfloat16).astype(np.float32))
#     return out

# %time coo_matmul(inverse_indices, sorted_indices, np.asarray(sorted_weights.view(np.float16)), np.asarray(weight_dec.view(np.float16)));

In [ ]:
%time index_array_np = np.asarray(indices.ravel().astype(np.uint32))
%time weight_array_np = np.asarray(weights.ravel().view(jnp.float16))
%time indices_to_sort = np.argsort(index_array_np);
%time inverse_indices = np.argsort(indices_to_sort);
%time sorted_indices = index_array_np[indices_to_sort];
%time sorted_weights = weight_array_np[indices_to_sort];

In [ ]:
# # Too slow
# import numba as nb

# @nb.njit
# def coo_matmul(inverse_indices, sorted_indices, sorted_weights, weights_dec):
#     N = inverse_indices.shape[0] // k
#     D = weights_dec.shape[1]
#     out = np.zeros((N, D), dtype=np.float32)
#     for i in range(len(inverse_indices)):
#         out[inverse_indices[i] // k] += (sorted_weights[i] * weights_dec[sorted_indices[i]])
#     return out

# %time result = coo_matmul(inverse_indices, sorted_indices, np.asarray(sorted_weights.view(jnp.bfloat16).astype(np.float32)), np.asarray(weight_dec.view(jnp.bfloat16).astype(np.float32)));

In [ ]:
import numba as nb

@nb.njit
def to_bf16(x):
    x = x.view(np.uint32)
    conv_lowbit = x >> 16 + (x & (1 << 15))
    return conv_lowbit.astype(np.uint16)

@nb.njit
def from_bf16(x):
    x = x.astype(np.uint32) << 16
    return np.ascontiguousarray(x.view(np.float32)[..., ::2])
    # return x.view(np.float32)
    # return (x.astype(np.uint32) << 16).view(np.float32)

print(to_bf16(np.array([200], dtype=np.float32)))
print(from_bf16(np.array([[12, 13, 14], [15, 16, 17]], dtype=np.uint16)))

In [ ]:
# # Too slow
# @nb.njit
# def coo_matmul(inverse_indices, sorted_indices, sorted_weights, weights_dec):
#     N = inverse_indices.shape[0] // k
#     D = weights_dec.shape[1]
#     out = np.zeros((N, D), dtype=np.uint16)
#     for i in range(len(inverse_indices)):
#         out[inverse_indices[i] // k] = to_bf16(from_bf16(out[inverse_indices[i] // k]) + (sorted_weights[i] * from_bf16(weights_dec[sorted_indices[i]])))
#     return out

# %time result = coo_matmul(inverse_indices, sorted_indices, np.asarray(sorted_weights.view(jnp.uint16)), np.asarray(weight_dec.view(jnp.uint16)));

In [ ]:
# from numba_progress import ProgressBar
%env OPENBLAS_NUM_THREADS=64

# @nb.njit(parallel=True)
# def sparse_matmul(indices, weights, weights_dec):
#     out = np.zeros((indices.shape[0], weights_dec.shape[1]), dtype=np.uint16)
#     for i in nb.prange(indices.shape[0]):
#         out[i] = to_bf16(from_bf16(weights[i]) @ from_bf16(weights_dec[indices[i]]))
#     return out


@nb.njit(nogil=True, parallel=True)
# def sparse_matmul(indices, weights, weights_dec, bar):
def sparse_matmul(indices, weights, weights_dec):
    out = np.zeros((indices.shape[0], weights_dec.shape[1]), dtype=np.uint16)
    for i in nb.prange(indices.shape[0]):
        out[i] = to_bf16(from_bf16(weights[i]) @ from_bf16(weights_dec[indices[i]]))
        # bar.update(1)
    # chunk = 64
    # for i_ in range(0, indices.shape[0], chunk):
        # for i in nb.prange(i_, min(i_ + chunk, indices.shape[0])):
        #     out[i] = to_bf16(from_bf16(weights[i]) @ from_bf16(weights_dec[indices[i]]))
        # bar.update(chunk)
    # return out

%time indices_np = np.asarray(indices.astype(np.uint32))
%time weights_np = np.asarray(weights.view(np.uint16))
%time weight_dec_np = np.asarray(weight_dec.view(np.uint16))
# with ProgressBar(total=indices_np.shape[0]) as bar:
#     result = sparse_matmul(indices_np, weights_np, weight_dec_np, bar)
%time result = sparse_matmul(indices_np, weights_np, weight_dec_np);

In [ ]:
# %time bcoo = jax.experimental.sparse.BCOO((jnp.asarray(sorted_weights).view(jnp.bfloat16), jnp.stack((jnp.asarray(inverse_indices // k), jnp.asarray(sorted_indices)), axis=-1).reshape(-1, 2)), shape=(weights.shape[0], weight_dec.shape[0])).block_until_ready();

In [ ]:
# import jax.experimental.sparse


# chunk_size = 2**12
# def sparse_matmul_cpu(row_indices, col_indices, weights, weight_dec):
#     def sparse_matmul_cpu_chunk(carry, chunk):
#         row_ind, col_ind, w = chunk
#         bcoo = jax.experimental.sparse.BCOO((w, jnp.stack((row_ind, col_ind), axis=-1).reshape(-1, 2)), shape=(weights.shape[0], weight_dec.shape[0]))
#         return carry + bcoo @ weight_dec, None
#     out = jnp.zeros((weights.shape[0], weight_dec.shape[1]), jnp.bfloat16)
#     return jax.lax.scan(sparse_matmul_cpu_chunk, out, tuple(x.reshape(-1, chunk_size) for x in (row_indices, col_indices, weights)))[0]

# sparse_matmul_cpu_jit = jax.jit(sparse_matmul_cpu, device=cpu)
# with jax.default_device(cpu):
#     %time result = sparse_matmul_cpu_jit(jnp.asarray(inverse_indices // k), jnp.asarray(sorted_indices), jnp.asarray(sorted_weights).view(jnp.bfloat16), weight_dec).block_until_ready();

In [ ]:
# import jax.experimental.sparse


# chunk_size = 2**12
# def sparse_matmul_cpu(row_ind, col_ind, w, weight_dec):
#     bcoo = jax.experimental.sparse.BCOO((w, jnp.stack((row_ind, col_ind), axis=-1).reshape(-1, 2)), shape=(weights.shape[0], weight_dec.shape[0]))
#     return bcoo @ weight_dec

# sparse_matmul_cpu_jit = jax.jit(sparse_matmul_cpu, device=cpu)
# with jax.default_device(cpu):
#     out = 0
#     for i in trange(0, len(sorted_indices), chunk_size):
#         out += sparse_matmul_cpu_jit(jnp.asarray(inverse_indices[i:i+chunk_size] // k), jnp.asarray(sorted_indices[i:i+chunk_size]), jnp.asarray(sorted_weights[i:i+chunk_size]).view(jnp.bfloat16), weight_dec)

In [ ]:
# from tqdm import trange
# masks = []
# chunked_weights = []
# chunked_indices = []
# for i in trange(0, w_size, w_chunk):
#     mask = (i < index_array_np) & (index_array_np < (i + w_chunk))
#     chunked_indices.append(index_array_np[mask] - i)
#     chunked_weights.append(weight_array_np[mask])
#     masks.append(np.nonzero(mask)[0])

In [ ]:
# from matplotlib import pyplot as plt
# plt.hist([x.shape[0] for x in chunked_indices])
# plt.show()

In [ ]:
b_chunk = 2**12
for idx in trange(0, w_size, b_chunk):
    sparse_matmul_out = sparse_matmul_jit(*(u[idx:idx + b_chunk] for u in encback), weight_dec)
    del sparse_matmul_out
    gc.collect()
# %time sparse_matmul_out = sparse_matmul_jit(*encback, weight_dec)

In [ ]:
encback[0].shape, encback[1].shape

In [ ]:
weights_dec[0].shape